In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig, AdamW, get_linear_schedule_with_warmup
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, EncoderDecoderModel, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.model_selection import train_test_split
data = pd.read_excel(r'/content/drive/MyDrive/bart/MeQSum_ACL2019_BenAbacha_Demner-Fushman.xlsx')

train_data, temp = train_test_split(data, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(temp, test_size=0.5, random_state=42)

train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "gpt2")



Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.8.ln_cross_attn.weight', 'h.1.crossattention.c_proj.bias', 'h.3.crossattention.q_attn.weight', 'h.0.ln_cross_attn.weight', 'h.3.crossattention.q_attn.bias', 'h.4.crossattention.c_attn.weight', 'h.8.crossattention.c_attn.bias', 'h.11.crossattention.q_attn.bias', 'h.8.crossattention.c_proj.weight', 'h.4.ln_cross_attn.bias', 'h.7.crossattention.q_attn.bias', 'h.6.crossattention.c_attn.weight', 'h.4.crossattention.q_attn.weight', 'h.1.crossattention.q_attn.weight', 'h.6.crossattention.c_attn.bias', 'h.9.crossattention.q_attn.bias', 'h.3.ln_cross_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.9.crossattention.c_attn.bias', 'h.2.ln_cross_attn.weight', 'h.9.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.4.crossattention.c_attn.bias', 'h.7.crossattention.c_proj.weight', 'h.5.crossattention.c_proj.bias', 'h.8.crossattention.c_attn.weight', 'h.3.crossatte

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, source_col, target_col, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_col = source_col
        self.target_col = target_col
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source = str(self.data[self.source_col][index])
        target = str(self.data[self.target_col][index])

        source = self.tokenizer.encode_plus(
            source,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt",
             truncation=True
        )

        target = self.tokenizer.encode_plus(
            target,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt",
             truncation=True
        )

        return {
            "input_ids": source["input_ids"].flatten(),
            "attention_mask": source["attention_mask"].flatten(),
            "labels": target["input_ids"].flatten(),
        }

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
train_dataset = CustomDataset(train_data, 'CHQ', 'Summary', tokenizer, 512)
val_dataset = CustomDataset(val_data, 'CHQ', 'Summary', tokenizer, 512)
test_dataset = CustomDataset(test_data, 'CHQ', 'Summary', tokenizer, 1000)

In [ ]:
decoder_start_token_id = tokenizer.cls_token_id
model.config.decoder.start_token_id = decoder_start_token_id
model.config.decoder_start_token_id = model.config.eos_token_id

In [ ]:
import numpy as np
best_loss = np.inf
best_batchsize = None
best_epoch = None

for batchsize in range(4,13,4):
    torch.cuda.empty_cache()
    train_dataloader = DataLoader(train_dataset, batch_size=batchsize)
    val_dataloader = DataLoader(val_dataset, batch_size=batchsize)

    optimizer = AdamW(model.parameters(), lr=1e-5)
    total_steps = len(train_dataloader) * 2
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(2):
        print(f'Epoch: {epoch+1}')

        # Training Phase
        model.train()
        total_train_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()

        # Validation Phase
        model.eval()
        total_val_loss = 0
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f'Validation Loss: {avg_val_loss}')

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_batchsize = batchsize
            best_epoch = epoch + 1
            model_path = "/content/drive/MyDrive/bart_research/best_model"
            model.save_pretrained(model_path)
            print(f'Saving model at epoch {best_epoch} with validation loss of {best_loss:.3f}')